**First, we import all the libraries that we will be using in this section**

In [2]:
import FeaturesExtracts
import os  
import numpy as np
import cv2
import mediapipe as mp
import math
import pandas as pd

#............................................


**A function that produces hand fetuers from images of the data set, based on the `FeaturesExtracts` code file, and saves these fetuers in a file.**

In [3]:
def Fetuers_Extraction_And_Save(Path_Dataset,Path_File_Fetuers):
    
    hands=mp.solutions.hands.Hands(static_image_mode=True,max_num_hands=1,min_detection_confidence=0.7)
    index=0
    errorfile=0
    fileobj=open(Path_File_Fetuers,'a')
    for folder in [fd for fd in os.listdir(Path_Dataset) if os.path.isdir(Path_Dataset+'\\'+fd)]:
        path=Path_Dataset+"\\"+folder
        for file in [(path+'\\'+fi) for fi in os.listdir(path) if os.path.isfile(path+'\\'+fi)]:
            
            try:
                
                image=cv2.imread(file)
                image= cv2.flip(image, 1)
                res=hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))  
                
                if res.multi_hand_landmarks:
                    IsLeftHand=res.multi_handedness[0].classification[0].label!="Right"
                    
                    Fetuers=FeaturesExtracts.HandFetuers.GetFeteuers(res.multi_hand_landmarks[0].landmark, IsLeftHand)
                    
                    FetuersAll=np.append(Fetuers,[[ord(folder)]],axis=1)    
                    np.savetxt(fileobj,FetuersAll,fmt='%f',delimiter=';')
                    index=index+1
                    print("Proccessing Images = ",index)
                    
                   
                    
            except BaseException:
                errorfile=errorfile+1
                print('errror......')
                continue
                 
      
    fileobj.close()
    print('.............................\n',"Correct files numbers= ",index)
    print("Error files numbers= ",errorfile)
    del hands
    
#............................................   
                

**The data set used in this project is pictures of English letters using sign language. All letters are used except for letters that depend on movement, which are (Z, J).**

**`The dataset is available at:` https://www.kaggle.com/datasets/debashishsau/aslamerican-sign-language-aplhabet-dataset**

In [ ]:
path_dataset=r"SigNN Character Database"
path_fetuers=r"..\data\data fetuers\Data Fetuers.txt"

Fetuers_Extraction_And_Save(Path_Dataset=path_dataset,Path_File_Fetuers=path_fetuers)

#............................................


**View extracted properties**

In [ ]:
df=pd.read_csv(path_fetuers,sep=';',header=None)
df[df.columns[-1]]=df[df.columns[-1]].map(lambda x:chr(int(x))).astype(str)
print("Dimensions of the data set = ",df.shape)
X=df.iloc[:,:-1]
Y=np.asarray(df.iloc[:,-1])

df.sample(10)

#............................................
